In [1]:
from utils import Predictor, arr2db, forna_url

Using TensorFlow backend.


In [3]:
# running on CPU is extremely slow!

In [4]:
import numpy as np

In [5]:
model = Predictor('model/2019_09_08.1/model.hdf5')

4


In [6]:
plot_model(model.model, to_file='/Users/alicegao/data//tmp/model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [ ]:
# tRNA

seq = 'GGGGATGTAGCTCATATGGTAGAGCGCTCGCTTTGCATGCGAGAGGCACAGGGTTCGATTCCCTGCATCTCCA'

pred, logp, fe = model.predict_one_step_ar(seq=seq, n_sample=20, start_offset=2)


In [ ]:
for idx_sample in range(pred.shape[0]):
    p = pred[idx_sample, :, :, 0]
    p[np.where(p==-1)] = 0
    db_str = arr2db(p)
    
    print idx_sample
    print logp[idx_sample]
#     print db_str
    print forna_url(seq, db_str)
    print ''